In [5]:
import psycopg2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt

import seaborn as sns
import datetime

## Outfielder DataFrames

In [6]:
#DF1=  clutch_outf_30.csv
#DF2 = clutch_outf_100.csv
#DF3 = allstr_clutch_outf.csv

In [7]:
import psycopg2 as p2
conn = p2.connect(database = 'player_stats', user ='baseball_master', password = 'georgetowndata', 
                  host = 'georgetown-baseball.cnfqonxqdbry.us-east-1.rds.amazonaws.com', 
                  port = '5432')


### Non- All Star Outfielders who have played at least 30 games at their position

In [ ]:
query="""
select "playerID", "POS", "yearID", "G"
from fielding 
where "yearID" >= '1947'
and "POS" = 'OF'
and "G" > 30 and "G" < 100
and "playerID" not in 
   (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5)
"""

Reg_Outfielder_30 = pd.read_sql_query(query, conn)

In [ ]:
Reg_Outfielder_30.head()

### Clutch - Impact on winning percentage 

In [8]:
query ="""

select "year" as "yearID", "batter_id" as "playerID", "event_count", "average_win_change", "center_weighted_win_change"
as "clutch_win_change"
from clutch_bat
order by "yearID"

"""

Clutch = pd.read_sql_query(query, conn)

In [9]:
Clutch.head()

,yearID,playerID,event_count,average_win_change,clutch_win_change
0,1950.0,aberal01,3.0,0.006172,-0.003889
1,1950.0,abramca01,55.0,0.010747,-0.006012
2,1950.0,addisbo01,24.0,0.014065,-0.013327
3,1950.0,bankhda01,41.0,0.005563,-0.005630
4,1950.0,bantaja01,14.0,0.010745,-0.015449


,yearID,playerID,event_count,average_win_change,clutch_win_change
0,1950.0,aberal01,3.0,0.006172,-0.003889
1,1950.0,abramca01,55.0,0.010747,-0.006012
2,1950.0,addisbo01,24.0,0.014065,-0.013327
3,1950.0,bankhda01,41.0,0.005563,-0.005630
4,1950.0,bantaja01,14.0,0.010745,-0.015449


### Merge Impact on Winning Percentage Stat with Player

In [ ]:
Clutch_Outfielder_30 =pd.merge(Clutch, Reg_Outfielder_30, on = ["playerID", "yearID"])

In [ ]:
Clutch_Outfielder_30.head()

### Offensive Stats

In [ ]:
query="""
select a."playerID", m."nameFirst", m."nameLast", m."birthYear",a."yearID", a."G", a."AB",a."H",a."2B", a."3B",
a."SO", a."HBP", a."BB", a."SF", a."R", a."HR", a."RBI"
from batting a
join master m
on a."playerID" = m."playerID"
where a."yearID" >= '1947'
"""

Batting_Stats = pd.read_sql_query(query,conn)

In [ ]:
Batting_Stats.head()

In [ ]:
Batting_Stats['AVG'] = Batting_Stats['H'] / Batting_Stats['AB']

In [ ]:
Batting_Stats["1B"] = Batting_Stats["H"] - Batting_Stats["2B"]- Batting_Stats["3B"]

In [ ]:
Batting_Stats.HBP.fillna(0, inplace = True)

In [ ]:
Batting_Stats.SF.fillna(0, inplace = True)

In [ ]:
Batting_Stats["TB"] = Batting_Stats["1B"] + Batting_Stats["2B"]*2 
+ Batting_Stats["3B"]*3 + Batting_Stats["HR"] * 4

In [ ]:
Batting_Stats["SLG"] = Batting_Stats["TB"] / Batting_Stats["AB"]

In [ ]:
Batting_Stats["OBP"] = (Batting_Stats["H"] + Batting_Stats["BB"] 
+ Batting_Stats["HBP"]) / (Batting_Stats ["AB"] + Batting_Stats ["BB"]
+ Batting_Stats["HBP"] + Batting_Stats["SF"])

In [ ]:
Batting_Stats["OPS"] = Batting_Stats["OBP"] + Batting_Stats["SLG"]

### Runs Scored = Measures how many runs a player help created offensive (directly or indirectly) for his team during the season 

In [ ]:
Batting_Stats["RC"] = Batting_Stats["TB"] * Batting_Stats["OBP"]

In [ ]:
Batting_Stats.head()

### Merge with other offensive stats

In [ ]:
pd.merge(Clutch_Outfielder_30, Batting_Stats, on = ["playerID", "yearID"])

In [ ]:
clutch_outf_30 = pd.merge(Clutch_Outfielder_30, Batting_Stats, on = ["playerID", "yearID"])

In [ ]:
clutch_outf_30.head(3)

In [ ]:
clutch_outf_30.dropna(how = 'any', inplace = True)

In [ ]:
clutch_outf_30.fillna(0).head(5)

In [ ]:
clutch_outf_30.drop('G_y', axis = 1, inplace = True)

In [ ]:
clutch_outf_30.head()

In [ ]:
clutch_outf_30.to_csv('clutch_outf_30.csv')

In [ ]:
clutch_outf_30 = pd.read_csv('clutch_inf_30.csv', index_col= 0 )

In [ ]:
clutch_outf_30.head() 

### Visualize data relationships using scatterplots

In [ ]:
##seaborn as sns
sns.pairplot(clutch_outf_30, x_vars = ['BB','SLG','HR','AVG'], y_vars = 'RC', height = 5, aspect = 0.7, kind = 'reg')

In [ ]:
sns.pairplot(clutch_outf_30, x_vars = ['TB','OPS','OBP','H'], y_vars = 'RC', height = 5, aspect = 0.7, kind = 'reg')

### Non- All Star Outfielders who have played at least 100 games at their position

In [ ]:
query="""
select "playerID", "POS", "yearID", "G"
from fielding 
where "yearID" >= '1947'
and "POS" = 'OF'
and "G" >= 100
and "playerID" not in 
   (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5)
"""

Reg_Outfielder_100 = pd.read_sql_query(query, conn)

### Merge Impact on Winning Percentage Stat with Player

In [ ]:
Clutch_Outfielder_100 =pd.merge(Clutch, Reg_Outfielder_100, on = ["playerID", "yearID"])

In [ ]:
Clutch_Outfielder_100.head()

### Merge with other offensive stats

In [ ]:
pd.merge(Clutch_Outfielder_100, Batting_Stats, on = ["playerID", "yearID"])

In [ ]:
clutch_outf_100 = pd.merge(Clutch_Outfielder_100, Batting_Stats, on = ["playerID", "yearID"])

In [ ]:
clutch_outf_100.head()

In [ ]:
clutch_outf_100 = pd.merge(Clutch_Outfielder_100, Batting_Stats, on = ["playerID", "yearID"])

In [ ]:
clutch_outf_100.dropna(how = 'any', inplace = True)

In [ ]:
clutch_outf_100.fillna(0).head(5)

In [ ]:
clutch_outf_100.drop('G_y', axis = 1, inplace = True)

In [ ]:
clutch_outf_100.head()

In [ ]:
clutch_outf_100.to_csv('clutch_outf_100.csv')

In [ ]:
clutch_outf_100 = pd.read_csv('clutch_outf_100.csv', index_col= 0 )

In [ ]:
clutch_outf_100.head() 

### Visualize data relationships using scatterplots

In [ ]:
##seaborn as sns
sns.pairplot(clutch_outf_100, x_vars = ['BB','SLG','HR','AVG'], y_vars = 'RC', height = 5, aspect = 0.7, kind = 'reg')

In [ ]:
sns.pairplot(clutch_outf_100, x_vars = ['TB','OPS','OBP','H'], y_vars = 'RC', height = 5, aspect = 0.7, kind = 'reg')

### All Star Outfielders who have played at least 100 games at their position

In [ ]:
query="""
select "playerID", "POS", "yearID","G"
from fielding 
where "yearID" >= '1947'
and "POS" = 'OF'
and "G" > 100
and "playerID" in  
        (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5)
"""

All_Star_Outfielder_100 = pd.read_sql_query(query, conn)

### Merge Impact on Winning Percentage Stat with Player

In [ ]:
Clutch_Allstr_Outfielder =pd.merge(Clutch, All_Star_Outfielder_100, on = ["playerID", "yearID"])

In [ ]:
Clutch_Allstr_Outfielder.head()

### Merge with other offensive stats

In [ ]:
allstr_clutch_outf = pd.merge(Clutch_Allstr_Outfielder, Batting_Stats, on = ["playerID", "yearID"])

In [ ]:
allstr_clutch_outf.head()

In [ ]:
allstr_clutch_outf.dropna(how = 'any', inplace = True)

In [ ]:
allstr_clutch_outf.fillna(0).head(5)

In [ ]:
allstr_clutch_outf.drop('G_y', axis = 1, inplace = True)

In [ ]:
allstr_clutch_outf.head()

In [ ]:
allstr_clutch_outf.to_csv('allstr_clutch_outf.csv')

In [ ]:
allstr_clutch_outf = pd.read_csv('allstr_clutch_outf.csv', index_col= 0 )

In [ ]:
allstr_clutch_outf.head() 

### Visualize data relationships using scatterplots

In [ ]:
##seaborn as sns
sns.pairplot(allstr_clutch_outf, x_vars = ['BB','SLG','HR','AVG'], y_vars = 'RC', height = 5, aspect = 0.7, kind = 'reg')

In [ ]:
sns.pairplot(allstr_clutch_outf, x_vars = ['TB','OPS','OBP','H'], y_vars = 'RC', height = 5, aspect = 0.7, kind = 'reg')